In [1]:
# Core
from pathlib import Path
import gzip
from warnings import warn
import json
import os
import time

# 3rd party
from wget import download
import numpy as np
import pandas as pd
import astropy.units as u

In [2]:
class MinorPlanetDB():
    def __init__(self,
                 path='../data/mpcorb_extended.json.gz',
                 download_url='http://minorplanetcenter.net/Extended_Files/mpcorb_extended.json.gz',
                 max_age=1 * u.day):
        
        self.path = Path(path)
        self.url = download_url
        self.max_age = max_age
        
        self.refresh_db()
                    
    def refresh_db(self):
        
        # Check if MPCORB database already downloaded. If not attempt to download it
        if not self.path.exists():
            warn("MPC Orbit Database file not found. Downloading...")
            download(self.url, out=self.path.as_posix())
            warn("Done.")
        # Check age of MPCORB database. If exceeds max_age download again
        elif (time.time() - os.path.getmtime(self.path.as_posix())) * u.second > self.max_age:
            warm("MPC Orbit Database file stale. Downloading a fresh copy...")
            download(self.url, out=self.path.as_posix())
            warn("Done.")
            
        # Load MPCORB database as a pandas DataFrame
        with gzip.open(self.path.as_posix(), 'rt') as db:
            self.db = pd.DataFrame(json.load(db))
            
        # Create a unique ID column from Number, Name (if present) or Principal_desig if not
        self.db['ID'] = (self.db.Number + ' ' + self.db.Name).combine_first(self.db.Principal_desig)
        
        # Set unique ID as the index
        self.db.set_index('ID', inplace=True)


In [3]:
def min_magnitude_estimate(minor_planet):
    return minor_planet.H + 2.5 * np.log(minor_planet.Perihelion_dist**2 * (1 - minor_planet.Perihelion_dist)**2)

In [4]:
minor_planets = MinorPlanetDB()

In [64]:
minor_planets.db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747208 entries, 0 to 747207
Data columns (total 38 columns):
Aphelion_dist                         747208 non-null float64
Arc_length                            139550 non-null float64
Arc_years                             607658 non-null object
Computer                              747208 non-null object
Critical_list_numbered_object_flag    578 non-null float64
Epoch                                 747208 non-null float64
G                                     743855 non-null float64
H                                     743855 non-null float64
Hex_flags                             747208 non-null object
Last_obs                              747208 non-null object
M                                     747208 non-null float64
NEO_flag                              16928 non-null float64
Name                                  21111 non-null object
Node                                  747208 non-null float64
Num_obs                        

In [7]:
minor_planets.db[np.logical_or(lambda x: x.Perihelion_dist < 1.0, lambda x: max_magnitude_estimate(x) < 12)]

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Ref,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms
ID,,,,,,,,,,,,,,,,,,,,,
(1566) Icarus,1.969502,NaN,1949-2015,MPCW,NaN,2458000.5,0.15,16.90,9803,2015-07-31,...,MPO350811,0.170525,9.375110,2.457965e+06,0,1.078096,0.826834,22.82427,0.880477,1.08
(1620) Geographos,1.662922,NaN,1951-2017,MPCW,NaN,2458000.5,0.15,15.60,9803,2017-01-11,...,MPO400009,0.552588,3.566728,2.458161e+06,0,1.245260,0.335401,13.33746,0.709274,1.78
(1685) Toro,1.963896,NaN,1948-2017,MPCW,NaN,2458000.5,0.15,14.23,1803,2017-04-17,...,MPO405857,0.553670,2.669154,2.457945e+06,0,1.367471,0.436152,9.38041,0.616349,0.48
(1862) Apollo,2.293034,NaN,1957-2017,MPCLINUX,NaN,2458000.5,0.09,16.25,9803,2017-05-17,...,MPO410096,0.504744,2.277956,2.458189e+06,0,1.470120,0.559759,6.35535,0.552936,0.50
(1863) Antinous,3.629322,NaN,1948-2017,MPCLINUX,NaN,2458000.5,0.15,15.54,1803,2017-01-30,...,MPO400012,0.714410,1.417327,2.457520e+06,0,2.259413,0.606312,18.39767,0.290209,0.52
(1864) Daedalus,2.358615,NaN,1971-2017,MPCLINUX,NaN,2458000.5,0.15,14.85,1803,2017-09-29,...,MPO416726,0.454765,2.305578,2.457959e+06,1,1.461004,0.614380,22.21052,0.558119,0.44
(1865) Cerberus,1.584363,NaN,1971-2017,MPCLINUX,NaN,2458000.5,0.15,16.84,1803,2017-10-01,...,MPO416726,0.422289,9.166392,2.458177e+06,1,1.080054,0.466929,16.09516,0.878084,0.56
(1866) Sisyphus,2.913099,NaN,1955-2016,MPCW,NaN,2458000.5,0.15,12.40,1803,2016-08-15,...,MPO416726,0.672107,1.622930,2.457773e+06,0,1.893391,0.538562,41.20212,0.378306,0.48
(1981) Midas,2.930709,NaN,1973-2015,MPCLINUX,NaN,2458000.5,0.15,15.20,9803,2015-02-24,...,MPO350815,0.512528,1.731699,2.458249e+06,1,1.775933,0.650237,39.83300,0.416451,0.45
